#Leitor de processos e Extrator de localizações e endereços
Esse programa utiliza a IA para extrair localizações e endereços de uma lista de processos, criando um arquivo .pkl que será enviado ao programa de transformação em latitude e longitude e seleção de locais de interesse.

In [20]:
#!python -m spacy download pt_core_news_sm
#!pip install PyPDF2

In [21]:
from google.colab import drive
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import spacy
import glob
import PyPDF2
import pickle

drive.mount('/content/drive')
nlp = spacy.load("pt_core_news_sm")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
cd drive/My Drive/Mestrado Unicamp/Hackaton CNJ

[Errno 2] No such file or directory: 'drive/My Drive/Mestrado Unicamp/Hackaton CNJ'
/content/drive/My Drive/Mestrado Unicamp/Hackaton CNJ


#Montar modelo, word_to_index_dict e word_to_index_dict_inverse

In [23]:
model = tf.keras.models.load_model('lstm_bidirectional.h5')
pickle_file = open("word_to_index_dict.pkl", "rb")
word_to_index_dict = pickle.load(pickle_file)
pickle_file.close()
word_to_index_dict_inverse =  {w: k for k, w in word_to_index_dict.items()}
def _index_list_to_text(index_list, word_index_inverse):
    return ' '.join([word_index_inverse.get(i, "") for i in index_list])

#Função que prepara os dados. 
Transforma uma lista de frases em uma lista de lista de índices

In [24]:
max_seq_len = 20
def prepararInput(lista_de_frases):
  lista_de_lista_de_palavras = [text_to_word_sequence(frase, filters = '!"“”#$%&()*+-/:=?@[\\]^_`{|}~\t\n',lower=True, split=' ') for frase in lista_de_frases]
  for i in range(len(lista_de_lista_de_palavras)):
    for j in range(len(lista_de_lista_de_palavras[i])):
      lista_de_lista_de_palavras[i][j] = re.sub("[0-9]+", 'cinco', lista_de_lista_de_palavras[i][j])

  lista_de_lista_de_indices = [[word_to_index_dict[palavra] if palavra in word_to_index_dict else word_to_index_dict['<UNK>'] for palavra in lista_de_palavras] for lista_de_palavras in lista_de_lista_de_palavras]

  x = lista_de_lista_de_indices

  x = pad_sequences(x, 
                    maxlen=max_seq_len, 
                    dtype='int32',
                    padding='pre', 
                    truncating='pre', 
                    value=word_to_index_dict['<PAD>'])
  return x

#Função que extrai os endereços
A função extrairEnderecos extrai todos os endereços de um texto e os retorna em uma lista.

Para fazer isso, ela extrai todas as possibilidades de span (conjuntos de palavras) em torno de uma entidade nomeada identificada pela IA do spaCy como LOC.

O tamanho do span é dado por LEFT_TOKENS e RIGHT_TOKENS.

Os spans são filtrados pela IA, apenas são mantidos os com output > MIN_OUTPUT. Depois, de todos os spans que são endereço, é selecionado o maior.



In [25]:
LEFT_TOKENS = 8
RIGHT_TOKENS = 15
MIN_OUTPUT = 0.95
def extrairEnderecos(texto):
  doc = nlp(texto)
  enderecos = []
  for ent in doc.ents:
    if ent.label_ == 'LOC' or ent.label_ == 'MISC':
      spans = []
      sizes = []
      for right_token in range(RIGHT_TOKENS):
        for left_token in range(LEFT_TOKENS):
          size = left_token + right_token + ent.end - ent.start
          if size < max_seq_len:
            span = doc[ent.start-left_token:ent.end+right_token].text
            if span not in spans:
              spans.append(span)
              sizes.append(size)
      
      if len(spans)>0:
        output = model.predict(prepararInput(spans))
      else:
        output = np.array([[]])
      #agora tenho 3 arrays de mesmo tamanho: spans, sizes e output.
      #A seguir vou manter apenas os elementos desses 3 arrays que contém acc > MIN_OUTPUT
      spans = np.array(spans)
      sizes = np.array(sizes)
      output = output.reshape((-1))
      s = output > MIN_OUTPUT
      spans = spans[s]
      sizes = sizes[s]
      output = output[s]
      
      if len(sizes)>0:
        #enderecos.append(spans[sizes.argmax()])  #pegar o maior span
        enderecos.append(spans[output.argmax()])  #pegar o melhor
  enderecos = list(dict.fromkeys(enderecos))
  return enderecos

##Exemplos

In [26]:
exemplos_positivos = ['RUA  GERALDO BARBOSA nº 75 FRENTE, Acrelândia',
                      'RUA  DOS PIONEIROS 2044, Acrelândia/AC / CEP 69945-000',
                      'RUA  JOSE ALVES DOS SANTOS, 146, Acrelândia/AC/ CEP 69945-000',
                      'NASCEU NO DIA 5 DO 10',
                      'RUA  PEDRO RODRIGUES, 50, Acrelândia - AC -CEP 69945-000',
                      'VAMOS TESTAR ISSO AQUI',
                      'RAMAL  GRANADA, 129 CCE, Acrelândia/AC - CEP 69945-000',
                      'O RATO ROEU A ROUPA DO REI',
                      'RAMAL  NOVO ENCANTO, 51 CCE, Acrelândia/ AC -CEP 69945-000']

In [27]:
extrairEnderecos(' '.join(exemplos_positivos))

['RUA  GERALDO BARBOSA nº 75 FRENTE, Acrelândia RUA  DOS PIONEIROS 2044, Acrelândia/AC',
 'RUA  DOS PIONEIROS 2044, Acrelândia/AC / CEP 69945-000 RUA  JOSE',
 'CEP 69945-000 RUA  JOSE ALVES DOS SANTOS, 146, Acrelândia/AC/ CEP 69945-000',
 'RUA  JOSE ALVES DOS SANTOS, 146, Acrelândia/AC/ CEP 69945-000',
 'SANTOS, 146, Acrelândia/AC/ CEP 69945-000',
 'SANTOS, 146, Acrelândia/AC/ CEP 69945-000 NASCEU',
 'CCE, Acrelândia/AC - CEP 69945-000',
 'RAMAL  NOVO ENCANTO, 51 CCE, Acrelândia/ AC -CEP 69945-000']

#Montando uma lista com acórdãos do TJSP extraídos pelo Arthur 

In [28]:
path = 'dados/tjsp/'
files = glob.glob(path+'*.*')
tjsp = []
tjsp_file_names = []
for file_name in files:
  with open(file_name, 'rb') as f:
    fileReader = PyPDF2.PdfFileReader(f)
    for i in range(fileReader.numPages):
      pageObj = fileReader.getPage(i)
      texto = pageObj.extractText()
      texto = texto.replace('\n','')
      texto = texto.replace(',', ' , ')
      texto = texto.replace(';', ' ; ')
      texto = texto.replace('.', ' . ')
      tjsp.append(texto)
      tjsp_file_names.append(file_name.split('/')[-1].split('.')[0])
print('len(tjsp) =', len(tjsp))

len(tjsp) = 468


In [29]:
# Criando as localizações
localizacoes = []
for pdf, file_name in zip(tjsp, tjsp_file_names):
  pdf_salvo = pdf
  locais = extrairEnderecos(pdf)
  localizacoes = localizacoes + [locais]

In [30]:
#Limpar as localizações vazias
localizacoes2 = []
tjsp_file_names2 = []
for locais,file_name in zip(localizacoes, tjsp_file_names):
  if len(locais)>0:
    localizacoes2.append(locais)
    tjsp_file_names2.append(file_name)

In [31]:
#Salvando no disco para testar na próxima etapa
localizacoes_dicio = {'localizacoes': localizacoes2, 'numero_dos_processos' : tjsp_file_names2}
pickle_file = open("enderecos_processos.pkl", "wb")
pickle.dump(localizacoes_dicio, pickle_file)
pickle_file.close()

#Próximas células carregam o pkl e testam

In [32]:
texto = tjsp[tjsp_file_names.index(tjsp_file_names2[1])]
#model.predict(prepararInput(texto))
extrairEnderecos(texto)

['Floriano Peixoto ,  1750 ,  JUIZADO ESPECIAL CÍVEL ,  CRIMINAL E FAZENDA PÚBLICA',
 'FAZENDA PÚBLICA - CentroCEP: 15130-007 - Mirassol',
 'Mirassol - SPTelefone: (17',
 'brProcesso',
 'manifestação do D',
 'Mirassol']

In [33]:
tjsp[1]

'TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULOCOMARCA DE MIRASSOLFORO DE MIRASSOLJUIZADO ESPECIAL CÍVEL E CRIMINALRua Floriano Peixoto ,  1750 ,  JUIZADO ESPECIAL CÍVEL ,  CRIMINAL E FAZENDA PÚBLICA - CentroCEP: 15130-007 - Mirassol - SPTelefone: (17) 3242-3001 - E-mail: mirassoljec@tjsp . jus . brProcesso nº 0000234-77 . 2020 . 8 . 26 . 0358 - p .  1DECISÃOProcesso nº:0000234-77 . 2020 . 8 . 26 . 0358 Classe - AssuntoAção Penal - Procedimento Sumaríssimo - Crimes contra o Meio Ambiente e o Patrimônio GenéticoAutor:Justiça PúblicaRéu:Wilson Sérgio Candido da SilvaJuiz(a) de Direito: Dr(a) .  Marcos TakaokaAutos nº 2018/000229Vistos . Ante a manifestação do D . Promotor de Justiça de fls . 139 ,  arquivem-se os autos ,  anotando-se a extinção da pena imposta ao(à) sentenciado(a) Wilson Sérgio Candido da Silva (histórico de partes) ,  e a baixa definitiva dos autos (mov .  22) . Comunique-se IIRGD e TRE .  Publique-se e intimem-se . Mirassol ,  28 de maio de 2021 . DOCUMENTO ASSINADO DIGITA

In [34]:
texto = ['TRF1']
model.predict(prepararInput(texto))

array([[0.95592904]], dtype=float32)

In [35]:

texto = ['vilhena089']
prepararInput(texto)

array([[929589, 929589, 929589, 929589, 929589, 929589, 929589, 929589,
        929589, 929589, 929589, 929589, 929589, 929589, 929589, 929589,
        929589, 929589, 929589, 929590]], dtype=int32)

In [36]:
pickle_file = open("enderecos_processos_JF.pkl", "rb")
dic = pickle.load(pickle_file)
pickle_file.close()
dic

{'localizacoes': [['reserva indígena Alto Turiaçu'],
  ['Aldeia Aikanã', 'Reserva Indígena Tubarão Latundê'],
  ['Espigão do Oeste', 'Oeste/RO'],
  ['Reserva Indígena Tubarão Latundê e dois d'],
  ['Reserva Indígena Igarapé Lourdes'],
  ['Malhada/BA', 'Fundação Cultural Palmares'],
  ['Terra Indígena Guarani'],
  ['índios Rikbaktsa'],
  ['Itaituba/PA', 'ITAITUBA', 'ITAITUBA/PA .  1'],
  ['Altamira',
   'Altamira/PA',
   'Itaituba',
   'Itaituba/PA passa',
   'Jacareacanga',
   'Itaituba/PA'],
  ['Rio Guamá (TIARG', 'Piriá/PA', 'Garrafão'],
  ['Norte/PA', 'Piriá/PA'],
  ['floresta amazônica'],
  ['interior de Reserva Indígena da região (TI Zoró ,  TI Suruí',
   'região (TI Zoró ,  TI Suruí'],
  ['Prado (BA',
   'Terra Indígena (TI',
   'Indígena (TI',
   'Pataxó',
   'Ilhéus',
   'Olivença',
   'Buerarema',
   'Terra Indígena Pataxó'],
  ['Pataxó',
   'Prado-BA',
   'RESERVA INDÍGENA CUMURUXATIBA',
   'indígena Tupinambá'],
  ['RESERVA INDÍGENA TUPINAMBÁ', 'IMÓVEL RURAL'],
  ['Altamira'

In [37]:
for locais, file_name in zip(localizacoes,tjsp_file_names):
  if len(locais) > 0:
    print(file_name)
    print(locais)

doc_62475066
['CRIMINALAV ESTADOS UNIDOS ,  480 ,  Osvaldo Cruz - SP - CEP 17700-000', 'Osvaldo Cruz - SP - CEP 17700-000', 'LUCAS RICARDO', 'GUIMARÃESVistos', 'LUCAS RICARDO GUIMARAES']
doc_59672608
['Floriano Peixoto ,  1750 ,  JUIZADO ESPECIAL CÍVEL ,  CRIMINAL E FAZENDA PÚBLICA', 'FAZENDA PÚBLICA - CentroCEP: 15130-007 - Mirassol', 'Mirassol - SPTelefone: (17', 'brProcesso', 'manifestação do D', 'Mirassol']
doc_73261456
['Rua Riozuke Aoki ,  Centro', 'Tapiraí', 'Piedade', 'D E C I', 'DALL AGLIO']
doc_73261456
['Narra', 'DALL AGLIO']
doc_73261456
['DALL AGLIO']
doc_67862835
['DA SILVA ,  571 ,  Ubatuba - SP - CEP 11680-000', 'Ubatuba - SP - CEP 11680-000', 'Rua Angico ,  fundos do nº 39 ,  Taquaral ,  n', 'Taquaral', 'Ubatuba', 'Floresta Alta de Restinga', 'FABRICIO JOSE']
doc_67862835
['DA SILVA ,  571 ,  Ubatuba - SP - CEP 11680-000', 'Ubatuba - SP - CEP 11680-000', 'Floresta Alta de Restinga', 'Rio Taquaral', 'BENEDITO MOREIRA', 'FABRICIO JOSE']
doc_67862835
['DA SILVA ,  571 ,  

In [38]:
import json
with open('enderecos_processos.pkl', 'rb') as f:
    data = pickle.load(f)
    localizacoes = data["localizacoes"]
    numero_dos_processos = data["numero_dos_processos"]
    processos = []
    for i, processo in enumerate(numero_dos_processos):
        if i < len(localizacoes):
            processos.append(
                {
                    "id": i,
                    'nrProcesso': processo,
                    "address": localizacoes[i]
                }
            )
    with open(f"processos.json", "w", encoding="utf-8") as file:
        json.dump(processos, file, ensure_ascii=False)
processos

[{'address': ['CRIMINALAV ESTADOS UNIDOS ,  480 ,  Osvaldo Cruz - SP - CEP 17700-000',
   'Osvaldo Cruz - SP - CEP 17700-000',
   'LUCAS RICARDO',
   'GUIMARÃESVistos',
   'LUCAS RICARDO GUIMARAES'],
  'id': 0,
  'nrProcesso': 'doc_62475066'},
 {'address': ['Floriano Peixoto ,  1750 ,  JUIZADO ESPECIAL CÍVEL ,  CRIMINAL E FAZENDA PÚBLICA',
   'FAZENDA PÚBLICA - CentroCEP: 15130-007 - Mirassol',
   'Mirassol - SPTelefone: (17',
   'brProcesso',
   'manifestação do D',
   'Mirassol'],
  'id': 1,
  'nrProcesso': 'doc_59672608'},
 {'address': ['Rua Riozuke Aoki ,  Centro',
   'Tapiraí',
   'Piedade',
   'D E C I',
   'DALL AGLIO'],
  'id': 2,
  'nrProcesso': 'doc_73261456'},
 {'address': ['Narra', 'DALL AGLIO'], 'id': 3, 'nrProcesso': 'doc_73261456'},
 {'address': ['DALL AGLIO'], 'id': 4, 'nrProcesso': 'doc_73261456'},
 {'address': ['DA SILVA ,  571 ,  Ubatuba - SP - CEP 11680-000',
   'Ubatuba - SP - CEP 11680-000',
   'Rua Angico ,  fundos do nº 39 ,  Taquaral ,  n',
   'Taquaral',
   'U